# Part 1

In [16]:
// inspired by https://stackoverflow.com/a/40453980
def splitDocs(lines: Stream[String]): Stream[String] = {
    if (lines.isEmpty) Stream.Empty
    else {
        val (doc, rest) = lines.span(_.trim.nonEmpty)
        doc.mkString(" ") #:: splitDocs(rest.dropWhile(_.trim.isEmpty))
    }
}

defined function splitDocs

In [18]:
val docLines = getDocs(io.Source.fromFile("input").getLines.toStream)

docLines: Stream[String] = Stream(
  "iyr:2010 ecl:gry hgt:181cm pid:591597745 byr:1920 hcl:#6b5442 eyr:2029 cid:123",
  "cid:223 byr:1927 hgt:177cm hcl:#602927 iyr:2016 pid:404183620 ecl:amb eyr:2020",
  "byr:1998 ecl:hzl cid:178 hcl:#a97842 iyr:2014 hgt:166cm pid:594143498 eyr:2030",
  "ecl:hzl pid:795349208 iyr:2018 eyr:2024 hcl:#de745c hgt:157cm",
  "hgt:159cm pid:364060467 eyr:2025 byr:1978 iyr:2018 cid:117 ecl:hzl hcl:#18171d",
  "hcl:#cfa07d ecl:amb iyr:2012 hgt:182cm cid:338 eyr:2020 pid:374679609 byr:1925",
  "eyr:2021 byr:1981 hcl:#623a2f cid:195 iyr:2010 pid:579769934 ecl:grn hgt:192cm",
  "byr:1970 ecl:oth eyr:2025 pid:409994798 iyr:2018 hgt:189cm",
  "hgt:153cm pid:817651329 iyr:2019 eyr:2029 hcl:#623a2f byr:1920 ecl:gry",
  "iyr:2011 ecl:amb hcl:#a97842 byr:1965 pid:648375525 eyr:2028 hgt:177cm cid:287",
  "iyr:2012 pid:369979235 hcl:#c0946f ecl:amb hgt:178cm",
  "byr:1927 ecl:brn hgt:178cm eyr:2026 hcl:#efcc98 iyr:2011 pid:770851101",
  "eyr:2028 ecl:oth cid:298 byr:1943

In [19]:
val docs = docLines.map(_.split(" ").map(f => {val a = f.split(":", 2); a(0) -> a(1)}).toMap)

docs: Stream[Map[String, String]] = Stream(
  Map(
    "hgt" -> "181cm",
    "hcl" -> "#6b5442",
    "ecl" -> "gry",
    "byr" -> "1920",
    "eyr" -> "2029",
    "cid" -> "123",
    "pid" -> "591597745",
    "iyr" -> "2010"
  ),
  Map(
    "hgt" -> "177cm",
    "hcl" -> "#602927",
    "ecl" -> "amb",
    "byr" -> "1927",
    "eyr" -> "2020",
    "cid" -> "223",
    "pid" -> "404183620",
    "iyr" -> "2016"
  ),
  Map(
    "hgt" -> "166cm",
    "hcl" -> "#a97842",
    "ecl" -> "hzl",
    "byr" -> "1998",
    "eyr" -> "2030",
    "cid" -> "178",
    "pid" -> "594143498",
    "iyr" -> "2014"
  ),
  Map(
    "hgt" -> "157cm",
    "hcl" -> "#de745c",
    "ecl" -> "hzl",
    "eyr" -> "2024",
    "pid" -> "795349208",
    "iyr" -> "2018"
  ),
...

In [26]:
val expectedFields = Set(
    "byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid",
//     "cid"
)

expectedFields: Set[String] = Set(
  "hgt",
  "hcl",
  "ecl",
  "byr",
  "eyr",
  "pid",
  "iyr"
)

In [29]:
docs.filter(d => expectedFields.subsetOf(d.keySet)).size

res28: Int = 254

# Part 2

In [65]:
val hgtRegex = raw"(\d+)(cm|in)".r
val eyeColors = Set("amb", "blu", "brn", "gry", "grn", "hzl", "oth")

val rules: Map[String, (String) => Boolean] = Map(
    // byr (Birth Year) - four digits; at least 1920 and at most 2002.
    "byr" -> {s => {val y = s.toInt; 1920 <= y && y <= 2002}},
    // iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    "iyr" -> {s => {val y = s.toInt; 2010 <= y && y <= 2020}},
    // eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    "eyr" -> {s => {val y = s.toInt; 2020 <= y && y <= 2030}},
    // hgt (Height) - a number followed by either cm or in:
    //    If cm, the number must be at least 150 and at most 193.
    //    If in, the number must be at least 59 and at most 76.
    "hgt" -> {
        case hgtRegex(n, u) => {
            val h = n.toInt; 
            u == "cm" && 150 <= h && h <= 193 || u == "in" && 59 <= h && h <= 76
        }
        case _ => false
    },
    // hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    "hcl" -> {s => s.matches("#[0-9a-f]{6}")},
    // ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    "ecl" -> {s => eyeColors.contains(s)},
    // pid (Passport ID) - a nine-digit number, including leading zeroes.
    "pid" -> {s => s.matches("\\d{9}")},
    // cid (Country ID) - ignored, missing or not.
)

def isValid(doc: Map[String, String]): Boolean = {
    rules.map{case (f, rule) => doc.get(f).map(rule).getOrElse(false)}.reduce(_ && _)
}

def getViolations(doc: Map[String, String]): Set[String] = {
    rules.filter{case (f, rule) => !doc.get(f).map(rule).getOrElse(false)}.map(_._1).toSet
}

hgtRegex: scala.util.matching.Regex = (\d+)(cm|in)
eyeColors: Set[String] = Set("oth", "brn", "hzl", "blu", "gry", "grn", "amb")
rules: Map[String, String => Boolean] = Map(
  "hgt" -> ammonite.$sess.cmd64$Helper$$Lambda$3379/2109343486@12963b61,
  "hcl" -> ammonite.$sess.cmd64$Helper$$Lambda$3380/1702978288@11e1209,
  "ecl" -> ammonite.$sess.cmd64$Helper$$Lambda$3381/2027307540@6531be89,
  "byr" -> ammonite.$sess.cmd64$Helper$$Lambda$3376/983847137@2aeea718,
  "eyr" -> ammonite.$sess.cmd64$Helper$$Lambda$3378/825340961@900cc17,
  "pid" -> ammonite.$sess.cmd64$Helper$$Lambda$3382/1157263763@787f1964,
  "iyr" -> ammonite.$sess.cmd64$Helper$$Lambda$3377/1182492194@10523514
)
defined function isValid
defined function getViolations

In [66]:
// docs.map(getViolations)
docs.filter(isValid).size

res65: Int = 184